In [ ]:
pip install datasets tokenizers

## Load dataset

In [13]:
import os
import math

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers import models, trainers

base_path = 'trainingdata'

In [ ]:
files = [
    f'{base_path}/{folder}/{file_name}' for folder in os.listdir(base_path)
    if os.path.isdir(f'{base_path}/{folder}')
    for file_name in os.listdir(f'{base_path}/{folder}')
]

In [14]:

dataset = load_dataset('text', data_files=files, split='train')
dataset = dataset.filter(lambda line: not line['text'].startswith('>'))

Resolving data files: 100%|██████████| 363/363 [00:00<00:00, 810288.64it/s]
Using custom data configuration default-36ba59f92a81909b
Reusing dataset text (/Users/synch/.cache/huggingface/datasets/text/default-36ba59f92a81909b/0.0.0/acc32f2f2ef863c93c2f30c52f7df6cc9053a1c2230b8d7da0d210404683ca08)
100%|██████████| 15938/15938 [00:19<00:00, 811.32ba/s]


In [15]:
dataset[0]

{'text': 'ACAAATTATGATGGAAGTATACACCCTATATGGTTCCAAGCCCTTAAAGAAGTCTTATCCAACTTGGGGTTTGACTCCAATTTAATAGACAGAATATGTAAGTCAAAGCACATTTTTAAAAACATGTATTATGAAGTTGAAGGAGGGGTA'}

In [16]:

def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset), batch_size):
        yield dataset[i:i + batch_size]["text"]

## Train tokenizer

In [17]:
tokenizer = Tokenizer(models.BPE(cache_capacity=1000))
trainer = trainers.BpeTrainer(show_progress=True, vocab_size=24000)

In [12]:

tokenizer.train_from_iterator(batch_iterator(batch_size=1000),
                              trainer=trainer,
                              length=math.ceil(len(dataset) / 1000))

In [31]:
tokenizer.encode('ACTGCTGACGTCA').tokens

['AC', 'GT', 'ATAG', 'AACT', 'TTC']

In [4]:
tokenizer.enable_padding()

In [32]:
tokenizer.save(f'{base_path}/gene_tokenizer.json')

In [34]:
tok = tokenizer.from_file(f'{base_path}/gene_tokenizer.json')